In [1]:
import os
import json
import random
import pandas as pd

from langchain_mcp_adapters.client import MultiServerMCPClient

from react_agent.src.agents.react_agent import ReActAgent
from react_agent.src.agents.triage import Triage

from react_agent.src.util.tools_fabric import ToolsFabric

from experiments.models.experiment_models import LabeledQAPairFacts, ExperimentResult
from experiments.util.fact_score.fact_scorer import FactScorer

In [2]:
USE_MCP = True
DEBUG_MODE = False

fact_scorer = FactScorer()
path_to_ressources = "./resources/01_facts/"

files = {
    "extensibility": "extensibility_assistance_facts.json",
    "malaysia": "malaysia_support_facts.json",
    "peppol": "peppol_support_facts.json",
    "all": "all_cases_facts.json",
}

In [3]:
file_path = os.path.abspath(path_to_ressources + files["all"])


with open(file_path, encoding="utf8") as f:
    data = json.load(f)
    data_set = [LabeledQAPairFacts(**item) for item in data]

In [4]:
# Select 3 random elements
random_data_set = random.sample(data_set, 2)

In [5]:
triage_agent = Triage()
execution_trail = ""
experiment_results: list[ExperimentResult] = []

for data_row in random_data_set:
    result = ExperimentResult(
        id=data_row.id,
        question=data_row.question,
        answer=data_row.answer,
        product=data_row.product,
        category=data_row.category,
        persona=data_row.persona,
        activity=data_row.activity,
        country=data_row.country,
    )
    print(f"Asking agent question with id: {data_row.id}")
    query_categorization = triage_agent.triage_user_message(
        user_message=data_row.question
    )
    if DEBUG_MODE:
        print(f"Query categorization: {query_categorization}")

    tools = ToolsFabric.get_tools_for_category(
        use_mcp=USE_MCP,
        configuration=query_categorization["category"],
    )

    if USE_MCP:
        async with MultiServerMCPClient(tools) as client:
            agent = ReActAgent(tool_list=client.get_tools())
            execution_trail = await agent.arun_agent_with_input(
                user_message=query_categorization["user_query"], debug=DEBUG_MODE
            )
    else:
        agent = ReActAgent(tool_list=tools)
        execution_trail = agent.run_agent_with_input(
            user_message=query_categorization["user_query"], debug=DEBUG_MODE
        )

    run_data = agent.get_execution_data()

    result.tools_used = run_data.tools_used
    result.excecution_time_seconds = run_data.excecution_time_seconds
    result.model_used = run_data.model_used
    result.tokens_consumed = run_data.tokens_consumed
    result.llm_call_count = run_data.llm_call_count
    result.facts = data_row.facts

    print("Finished agent execution")

    print("Started FactScore calculation")
    result.fact_score = await fact_scorer.get_fact_score(
        facts=data_row.facts,
        knowledge_source=run_data.final_output,
        debug=DEBUG_MODE,
    )

    print(f"Finished experiment for id: {data_row.id}\n")

    experiment_results.append(result)

Asking agent question with id: EA-43
Finished agent execution
Started FactScore calculation
Finished experiment for id: EA-43

Asking agent question with id: EA-23
Finished agent execution
Started FactScore calculation
Finished experiment for id: EA-23



In [6]:
records = [r.model_dump(mode="json") for r in experiment_results]
df = pd.json_normalize(records)

df

/Users/i589287/Library/Caches/pypoetry/virtualenvs/react-agent-CUxtcVGL-py3.13/lib/python3.13/site-packages/pydantic/main.py:426: UserWarning: Pydantic serializer warnings:
  Expected `float` but got `FactScoreResult` with value `FactScoreResult(direct_fa... further processing.')])` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
/Users/i589287/Library/Caches/pypoetry/virtualenvs/react-agent-CUxtcVGL-py3.13/lib/python3.13/site-packages/pydantic/main.py:426: UserWarning: Pydantic serializer warnings:
  Expected `float` but got `FactScoreResult` with value `FactScoreResult(direct_fa...DOC_CHANGE_EMAIL_V2.')])` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


,id,question,answer,product,category,persona,activity,country,facts,bert_score,...,model_used,llm_call_count,fact_score.direct_fact_score,fact_score.supporting_fact_score,fact_score.combined_fact_score,fact_score.direct_facts,fact_score.supporting_facts,tokens_consumed.input_tokens,tokens_consumed.output_tokens,tokens_consumed.total_tokens
0,EA-43,What are the settings required before I can se...,Before you can start exchanging electronic doc...,None,None,None,None,None,[{'fact': 'You must define external document s...,0.0,...,gpt-4o,0,0.0,0.5,0.142857,[{'fact': 'You must define external document s...,[{'fact': 'Required settings are necessary to ...,19455,1071,20526
1,EA-23,What are the required steps to implement Busin...,To implement the Business Add-In (BAdI) EDOC_P...,None,None,None,None,None,[{'fact': 'The BAdI to implement is EDOC_PART_...,0.0,...,gpt-4o,7,1.0,1.0,1.000000,[{'fact': 'The BAdI to implement is EDOC_PART_...,[{'fact': 'The path to ES_EDOCUMENT_CLOUD is p...,19455,1071,20526
